# Data Mining Homework 4

## Problem 4

In [1]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

data4 = {'outlook': ['sunny','sunny','overcast','rainy','rainy','rainy','overcast','sunny','sunny','rainy','sunny','overcast','overcast','rainy'],
        'temp': [85,80,83,70,68,65,64,72,69,75,75,72,81,71],
        'humidity': [85,90,86,96,80,70,65,95,70,80,70,90,75,91],
        'windy': ['false','true','false','false','false','true','true','false','false','false','true','true','false','true'],
        'play' : ['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']
        }

df = pd.DataFrame (data4, columns = ['outlook','temp','humidity','windy','play'])

In [2]:
df

,outlook,temp,humidity,windy,play
0,sunny,85,85,false,no
1,sunny,80,90,true,no
2,overcast,83,86,false,yes
3,rainy,70,96,false,yes
4,rainy,68,80,false,yes
5,rainy,65,70,true,no
6,overcast,64,65,true,yes
7,sunny,72,95,false,no
8,sunny,69,70,false,yes
9,rainy,75,80,false,yes


Part 1 Using Entropy/Information Gain

In [3]:
def find_entropy(df):
    Class = df.keys()[-1]
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

In [4]:
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]
    target_variables = df[Class].unique()
    variables = df[attribute].unique()
    entropy_attribute1 = 0
    for variable in variables:
        entropy_individual = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] == target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy_individual += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy_attribute1 += -fraction2*entropy_individual
    return abs(entropy_attribute1)

In [5]:
def find_highest(df):
    Entropy_val = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]   

In [6]:
def create_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop = True)

In [7]:
import sys
sys.setrecursionlimit(10000)

In [8]:
def buildTree(df, tree = None):
    Class = df.keys()[-1]
    node = find_highest(df)
    attValue = np.unique(df[node])
    if tree is None:                    
        tree={}
        tree[node] = {}
    for value in attValue:
        
        subtable = create_subtable(df,node,value)
        clValue,counts = np.unique(subtable['play'],return_counts=True) 
        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable)  
   
    return tree

In [9]:
tree = buildTree(df)

In [10]:
import pprint
pprint.pprint(tree)

{'temp': {64: 'yes',
          65: 'no',
          68: 'yes',
          69: 'yes',
          70: 'yes',
          71: 'no',
          72: {'outlook': {'overcast': 'yes', 'sunny': 'no'}},
          75: 'yes',
          80: 'no',
          81: 'yes',
          83: 'yes',
          85: 'no'}}


In [11]:
def predict(inst,tree):
    for nodes in tree.keys():
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;
    return prediction

In [12]:
data2 = {'outlook': ['overcast'], 'temp': [64],'humidity': [62],'windy': ['false']}
df2 = pd.DataFrame (data2, columns = ['outlook','temp','humidity','windy'])
inst = df2.iloc[0]
inst

outlook     overcast
temp              64
humidity          62
windy          false
Name: 0, dtype: object

In [13]:
prediction = predict(inst,tree)

In [14]:
prediction

'yes'

## Part 2

In [15]:
import collections

def gini(df):
    total = len(df)
    colNames = []
    for col in df.columns:
        colNames.append(col)
    colNames = colNames[:-1]
    ginivalues = {}
    for i in colNames:
        temp = attsplit(df,i)
        tempValues = []
        for key in temp.keys():
            tempValues.append(temp[key] / total)
        coef = len(tempValues)
        val = 1
        for x in tempValues:
            val = val * x
        gini = coef*val
        ginivalues.update({i : gini})
    return ginivalues  

def attsplit(df, value):
    return dict(collections.Counter(df[value]))

def find_min_gini(df):
    temp = gini(df)
    minval = min(temp, key = temp.get)
    return minval

In [16]:
def buildTree2(df, tree = None):
    Class = df.keys()[-1]
    node = find_min_gini(df)
    attValue = np.unique(df[node])
    if tree is None:                    
        tree={}
        tree[node] = {}
    for value in attValue:
        
        subtable = create_subtable(df,node,value)
        clValue,counts = np.unique(subtable['play'],return_counts=True) 
        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable)  
   
    return tree

In [17]:
tree2 = buildTree2(df)

In [18]:
tree2

{'temp': {64: 'yes',
  65: 'no',
  68: 'yes',
  69: 'yes',
  70: 'yes',
  71: 'no',
  72: {'outlook': {'overcast': 'yes', 'sunny': 'no'}},
  75: 'yes',
  80: 'no',
  81: 'yes',
  83: 'yes',
  85: 'no'}}

In [19]:
prediction = predict(inst,tree2)
prediction

'yes'

Decision tree is same, predicts that in the sample case the value of play will be 'yes'